In [4]:
import importlib
import model.trainer
import torch
importlib.reload(model.trainer)
trainer = model.trainer.Trainer(
    # device=torch.device("cuda"),
    checkpoint=5
)

Using default device: cpu
Using default model
Using default vocoder


In [5]:
dataset = model.trainer.get_dataset()

In [6]:
losses = trainer.train_step(dataset[0])
model.trainer.Trainer.show_loss(0, 0, *losses)